# WS 12 AutoML with AutoGluon Hands on Module


## 1. Introduction
In this hands on module, we will see how to simplify the process of training high-quality, optimized machine learning models on sample datasets from UCI Machine Learning repository using the [AutoGluon](https://auto.gluon.ai/stable/index.html) package.

## 2. Dataset introduction and loading data from UCI ML Repository
Now we import pacakges and load in the following three healthcare related datasets from [UCI Machine Learning Repository](https://archive.ics.uci.edu/)


*   [Breast Cancer data](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic) from University of Wisconsin
*   [Diabetes data](https://archive.ics.uci.edu/dataset/296/diabetes+130-us+hospitals+for+years+1999-2008) representing ten years of clinical care at 130 US hospitals
*  [A Drug reviews](https://archive.ics.uci.edu/dataset/461/drug+review+dataset+druglib+com) dataset providing patient reviews on specific drugs

The first two datsets will be used to demonstrate AutoGluon's `TabularPredictor` class and how it enables us to train high-fideltiy ensemble models on data without needing to worry about pre-processing.

The third dataset will allow us to explore AutoGluon's `MultiModalPredictor` and how it allows us to train models on plain-text inputs









In [1]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo

In [2]:
# here we fix a random seed for reproducibility purposes
np.random.seed(913)

Now we load in the breast cancer dataset. This dataset contains features that describe the characteristics of cell nuclei present in a digitized image taken from the fine needle aspirate of a breast mass. The labels in the data are binary/two-class, with 'B' representing a benign mass and 'M' representing a malignant mass



In [3]:
# now we load in the breast cancer dataset from UCI
breast_cancer_wisconsin_diagnostic = fetch_ucirepo(id=17)

# data (as pandas dataframes)
X = breast_cancer_wisconsin_diagnostic.data.features
y = breast_cancer_wisconsin_diagnostic.data.targets


In [4]:
print(breast_cancer_wisconsin_diagnostic.variables)

                  name     role         type demographic description units  \
0                   ID       ID  Categorical        None        None  None   
1            Diagnosis   Target  Categorical        None        None  None   
2              radius1  Feature   Continuous        None        None  None   
3             texture1  Feature   Continuous        None        None  None   
4           perimeter1  Feature   Continuous        None        None  None   
5                area1  Feature   Continuous        None        None  None   
6          smoothness1  Feature   Continuous        None        None  None   
7         compactness1  Feature   Continuous        None        None  None   
8           concavity1  Feature   Continuous        None        None  None   
9      concave_points1  Feature   Continuous        None        None  None   
10           symmetry1  Feature   Continuous        None        None  None   
11  fractal_dimension1  Feature   Continuous        None        

In [5]:
breast_cancer_df = X.assign(
    Diagnosis=y
)

In [6]:
breast_cancer_df

,radius1,texture1,perimeter1,area1,smoothness1,compactness1,concavity1,concave_points1,symmetry1,fractal_dimension1,...,texture3,perimeter3,area3,smoothness3,compactness3,concavity3,concave_points3,symmetry3,fractal_dimension3,Diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,M
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,M
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,M
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,M
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,M
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,M
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,M
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,M


In [7]:
breast_cancer_df['Diagnosis'].value_counts()

Diagnosis
B    357
M    212
Name: count, dtype: int64

Next we load in the Diabetes dataset. This dataset was constructed with the goal of predicting the early readmission of diabetes patients within 30 days of discharge

In [8]:
# fetch dataset
diabetes_data = fetch_ucirepo(id=296)

# data (as pandas dataframes)
X = diabetes_data.data.features
y = diabetes_data.data.targets

/Users/troy/arch_workshop_automl_ws14/venv/lib/python3.10/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [12]:
print(diabetes_data.variables)

                        name     role         type demographic  \
0               encounter_id       ID                     None   
1                patient_nbr       ID                     None   
2                       race  Feature  Categorical        Race   
3                     gender  Feature  Categorical      Gender   
4                        age  Feature  Categorical         Age   
5                     weight  Feature  Categorical        None   
6          admission_type_id  Feature  Categorical        None   
7   discharge_disposition_id  Feature  Categorical        None   
8        admission_source_id  Feature  Categorical        None   
9           time_in_hospital  Feature      Integer        None   
10                payer_code  Feature  Categorical        None   
11         medical_specialty  Feature  Categorical        None   
12        num_lab_procedures  Feature      Integer        None   
13            num_procedures  Feature      Integer        None   
14        

In [13]:
y

,readmitted
0,NO
1,>30
2,NO
3,NO
4,NO
...,...
101761,>30
101762,NO
101763,NO
101764,NO


In [14]:
diabetes_df = X.assign(
    readmitted=y.map(lambda readmit: 0 if readmit == 'NO' else 1) # convert to a binary target
)

In [15]:
diabetes_df['readmitted'].value_counts()

readmitted
0    54864
1    46902
Name: count, dtype: int64

In [16]:
# because this is such a large dataset, we will down-sample this to only include 20% of the dat
diabetes_df_downsamp = diabetes_df.sample(frac=0.2)

In [17]:
diabetes_df_downsamp['readmitted'].value_counts()

readmitted
0    10912
1     9441
Name: count, dtype: int64

Now we split the two datasets into 80%/20% training/test set splits, so that we can evaluate our tuned models at the very end on unseen test data

In [18]:
bc_train = breast_cancer_df.sample(frac=0.8)
bc_test = breast_cancer_df.drop(bc_train.index)

In [19]:
bc_train['Diagnosis'].value_counts(normalize=True)

Diagnosis
B    0.608791
M    0.391209
Name: proportion, dtype: float64

In [20]:
bc_test['Diagnosis'].value_counts(normalize=True)

Diagnosis
B    0.701754
M    0.298246
Name: proportion, dtype: float64

In [21]:
diabetes_train = diabetes_df_downsamp.sample(frac=0.8)
diabetes_test = diabetes_df_downsamp.drop(diabetes_train.index)

In [22]:
diabetes_train['readmitted'].value_counts(normalize=True)

readmitted
0    0.535684
1    0.464316
Name: proportion, dtype: float64

In [23]:
diabetes_test['readmitted'].value_counts(normalize=True)

readmitted
0    0.537951
1    0.462049
Name: proportion, dtype: float64

## 3. Introduction to AutoGluon Tabular Predictor
Now we will see how AutoGluon's `TabularPredictor` class can be used to automatically fit a weighted ensemble on the breast cancer dataset, with automatic K-fold cross validation, bagging, and stacking, and with a large suite of models evaluated for inclusion in the final ensemble

In [24]:
from autogluon.tabular import TabularPredictor

In [25]:
# here we fit a tabular predictor on the breast cancer dataset
predictor_bc = TabularPredictor( # construct the predictor
    label='Diagnosis', eval_metric='roc_auc' # specify 'Diagnosis' as the label/target, and AUROC as the performance metric
).fit( # call the fit method
    bc_train,
    num_bag_folds=3, # specifies how many folds to run k-Fold cross validation with
    excluded_model_types=['NN_TORCH', 'FASTAI', 'CAT'] # exclude neural nets and CatBoost for faster training
)

No path specified. Models will be saved in: "AutogluonModels/ag-20260219_192117"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.10.16
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Wed Nov  5 21:32:34 PST 2025; root:xnu-11417.140.69.705.2~1/RELEASE_ARM64_T6020
CPU Count:          12
Pytorch Version:    2.9.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       55.74 GB / 96.00 GB (58.1%)
Disk Space Avail:   509.52 GB / 926.35 GB (55.0%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme'  : New in v1.5: The state-o

In [ ]:
# Now we evaluate the Breast Cancer model on the test data
predictor_bc.evaluate(bc_test)

{'roc_auc': np.float64(1.0),
 'accuracy': 0.9912280701754386,
 'balanced_accuracy': np.float64(0.99375),
 'mcc': 0.9794313218831192,
 'f1': 0.9855072463768116,
 'precision': 0.9714285714285714,
 'recall': 1.0}

*** SIGTERM received at time=1771528943 ***
PC: @        0x19e037d04  (unknown)  kevent
    @        0x318d07a2c  (unknown)  absl::lts_20230802::AbslFailureSignalHandler()
    @        0x19e0ad6a4  (unknown)  _sigtramp
    @        0x10516cb68  (unknown)  select_kqueue_control_impl
    @        0x104a7259c  (unknown)  method_vectorcall_FASTCALL
    @        0x104b78e90  (unknown)  _PyEval_EvalFrameDefault
    @        0x104a63320  (unknown)  _PyFunction_Vectorcall
    @        0x104b78e90  (unknown)  _PyEval_EvalFrameDefault
    @        0x104a63320  (unknown)  _PyFunction_Vectorcall
    @        0x104b78e90  (unknown)  _PyEval_EvalFrameDefault
    @        0x104a63320  (unknown)  _PyFunction_Vectorcall
    @        0x104b78e90  (unknown)  _PyEval_EvalFrameDefault
    @        0x104a63320  (unknown)  _PyFunction_Vectorcall
    @        0x104b78e90  (unknown)  _PyEval_EvalFrameDefault
    @        0x104a63320  (unknown)  _PyFunction_Vectorcall
    @        0x104b78e90  (unknown)  _PyEva

## 4. Comparison with Scikit Learn Toy Implementation of Bagging + Stacking
We see that our accuracy and precision rival that of the benchmark models listed on the UCI Machine Learning repository page for this dataset.


Now, for illustrative purposes, we will take a brief look at how much code it would take to implement a similar *(highly simplified)* k-fold bagging + stacking model ensembling such as what AutoGluon does automatically using Scikit Learn, another popular machine learning framework for Python


In [ ]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from xgboost import XGBClassifier
from tqdm import tqdm

In [ ]:
# our target labels are text character 'M' and 'B'
# Scikit-learn binary classifiers need these to be converted to numeric 1/0
bc_train_binary = bc_train.assign(
    binary_label=lambda x: x['Diagnosis'].map(lambda diag: 1 if diag == 'M' else 0)
).drop(columns='Diagnosis')
bc_test_binary = bc_test.assign(
    binary_label=lambda x: x['Diagnosis'].map(lambda diag: 1 if diag == 'M' else 0)
).drop(columns='Diagnosis')

In [ ]:
# separate features X from targets y
X = bc_train_binary.drop(columns=['binary_label'])
y = bc_train_binary['binary_label']
# initialize the Kfold object for doing kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# construct arrays for storing the out of fold prediciotns for the models
oof_preds_rf = np.zeros(len(X))
oof_preds_xgb = np.zeros(len(X))

# save the bagged models in lists

# specifiy the classifiers that will be in each layer
layers = [RandomForestClassifier, XGBClassifier]
layer_preds = [oof_preds_rf, oof_preds_xgb]
layer_bags = [list(), list()]
# loop over our layers
for i, layer in enumerate(layers):
  print(f"Performing k-fold cross validation at layer {i} with {layer}")
  # do the K-fold cross validation loop
  for train_idx, val_idx in tqdm(kf.split(X), total=5):
      # split inputs and outputs into training and validation
      X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
      y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

      # if we are in a layer past the first layer, the inputs need to include
      # the predictions from the prior layer
      if i > 0:
        X_train = np.column_stack([
            X_train.to_numpy(),
            layer_preds[i-1][train_idx] # include preds from prior layer
        ])
        X_val = np.column_stack([
            X_val.to_numpy(),
            layer_preds[i-1][val_idx]
        ])

      # fit a model from the given layer on the training fold
      model = layer()
      model.fit(X_train, y_train)
      # evaluate it on the validation fold and save oof predictions
      layer_preds[i][val_idx] = model.predict_proba(X_val)[:, 1]

      # save the model in our layer bag
      layer_bags[i].append(model)

# final meta model: Weighted ensemble of the predictions from the prior layers
meta_features = np.column_stack(layer_preds)

meta_model = LogisticRegression()
meta_model.fit(meta_features, y)

Now to evaluate this custom Meta model on the unseen test data, we need to


1.   Get 5 separate sets of predictions from each Random Forest model in the first layer
2.   Get 5 separate sets of predictions from each XGBoost model in the second layer, appending the predictions from the first layer models as feature inputs to the second layer models
3. Average the predicions made at each layer into a single set of predictions per layer
4. Append these two sets of predictions together into the final features to feed to the Meta Model (Logistic Regression)
5. Get the final predictions from the Meta Model



In [ ]:
X_test = bc_test_binary.drop(columns=['binary_label'])
y_test = bc_test_binary['binary_label']

layer_preds_test = []

# steps 1 and 2 - Getting separate predictions from models in each layer
for i in range(len(layer_bags)):
  # concatenate predictions from prior layer to features if needed
  if i > 0:
    X_test_stacked = np.column_stack([X_test.to_numpy(), layer_preds_test[i-1]])
  else:
    X_test_stacked = X_test

  # step 3 - compute average predictions across all models in the layer
  current_layer_preds = sum([
      model.predict_proba(X_test_stacked)[:, 1]
      for model in layer_bags[i]
  ]) / len(layer_bags[i])

  # save the predictions for the layer
  layer_preds_test.append(current_layer_preds)

# Step 4 - concatenate layer predicitons
meta_features_test = np.column_stack(layer_preds_test)

# Step 5 - get final predictions
final_predictions = meta_model.predict(meta_features_test)

# get the accuracy and f1 score
print(f"F1: {f1_score(y_test, final_predictions)}")
print(f"Accuracy: {accuracy_score(y_test, final_predictions)}")


## 5. Applying AutoGluon to a more complex dataset and threshold tuning
We can see that a considerable amount of code, understanding, and index manipulation was required in order to get an ensemble implementation using Scikit-learn that approaches the functionality which AutoGluon provides in only 3-4 lines of code. The toy implementation above also does not perform any greedy model weighting or pruning of models, and only incorporated two model families in the ensemble.

Here we train another TabularPredictor on the Diabetes readmission data, to assess how well it scales to larger datasets with heterogeneous input types. Here we will also see how one can tune the decision threshold of a binary classifier with AutoGluon in order to try and achieve better Recall and performance on the positive class

In [ ]:
diabetes_train

In [ ]:
predictor_diabetes = TabularPredictor( # construct the predictor
    label='readmitted', eval_metric='roc_auc'
).fit( # call the fit method
    train_data=diabetes_train,
    excluded_model_types=['NN_TORCH', 'FASTAI', 'CAT'], # excluding neural nets for faster training
)

In [ ]:
predictor_diabetes.evaluate(diabetes_test)

The recall of 0.54 tells us that only ~54% of patients who were readmitted to a hospital were successfully classified as such by our model. If we want to tune our classification threshold to achieve a better recall/sensitivity/TPR (true-positive rate), as we often do when working with medical data and developing tests for the presence of risks/conditions, AutoGluon makes this very easy.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, classification_report

In [ ]:
diabetes_preds = predictor_diabetes.predict(diabetes_test)

In [ ]:
ConfusionMatrixDisplay.from_predictions(diabetes_test['readmitted'], diabetes_preds)

In [ ]:
# now we calibrate the decision threshold of our model, using the F1 score as the calibration metric
threshold = predictor_diabetes.calibrate_decision_threshold(metric='f1')

Conceptual Question - Why did we calibrate the decision threshold to optimize the [F1 score](https://en.wikipedia.org/wiki/F-score) (the harmonic mean of precision and recall) if our goal was to achieve higher recall? Why not just use recall as the calibration metric directly?

In [ ]:
predictor_diabetes.set_decision_threshold(threshold)

In [ ]:
predictor_diabetes.evaluate(diabetes_test)

In [ ]:
diabetes_preds = predictor_diabetes.predict(diabetes_test)

In [ ]:
ConfusionMatrixDisplay.from_predictions(diabetes_test['readmitted'], diabetes_preds)


## 6. Tackling a Text Classification Problem with AutoGluon Tabular and MultiModal
Now that we will see how AutoGluon can also seamlessly tackle plain-text columns, such as written reviews, using both the `TabularPredictor` class, as well as the more sophisticated `MultiModalPredictor` which trains a full transformer neural network to make predictions on text-based data.

In [ ]:
from autogluon.multimodal import MultiModalPredictor

In [ ]:
# now we load in the Drug Reviews dataset from UCI Machine learning repository
drug_reviews_druglib_com = fetch_ucirepo(id=461)

# data (as pandas dataframes)
X = drug_reviews_druglib_com.data.features

In [ ]:
print(drug_reviews_druglib_com.variables)

In [ ]:
X

In [ ]:
# construct a 3-class label based on the numeric rating from 1-10
targets = X['rating'].map(lambda rating: 'positive' if rating >= 7 else 'neutral' if rating >= 4 else 'negative')

In [ ]:
drug_reviews_df = X[['benefitsReview', 'sideEffectsReview', 'commentsReview']].assign(
    target=targets
)

In [ ]:
drug_reviews_df

In [ ]:
drug_reviews_df['target'].value_counts()

In [ ]:
# we will downsample the three classes to achieve a smaller dataset for demonstration purposes
drug_reviews_downsamp = pd.concat([
    drug_reviews_df.query('target == "positive"').sample(n=400),
    drug_reviews_df.query('target == "neutral"').sample(n=400),
    drug_reviews_df.query('target == "negative"').sample(n=400)
])

In [ ]:
drug_reviews_train = drug_reviews_downsamp.sample(frac=0.8)
drug_reviews_test = drug_reviews_downsamp.drop(drug_reviews_train.index)

In [ ]:
drug_reviews_train['target'].value_counts(normalize=True)

In [ ]:
drug_reviews_test['target'].value_counts(normalize=True)

In [ ]:
# first we fit a TabularPredictor to the dataset to see how it performs
predictor_tab = TabularPredictor(label='target', eval_metric='acc')

In [ ]:
predictor_tab.fit(drug_reviews_train)

In [ ]:
# now we evaluate the model's performance on the test data
predictor_tab.evaluate(drug_reviews_test)

In [ ]:
ConfusionMatrixDisplay.from_predictions(drug_reviews_test['target'], predictor_tab.predict(drug_reviews_test))

In [ ]:
print(classification_report(drug_reviews_test['target'], predictor_tab.predict(drug_reviews_test)))

In [ ]:
# now we construct our model using the MultiModalPredictor class
predictor = MultiModalPredictor(label='target', eval_metric='acc')

In [ ]:
import gc

del(predictor_tab)
del(predictor_bc)
del(predictor_diabetes)

gc.collect()

In [ ]:
predictor.fit(drug_reviews_train, time_limit=180)

In [ ]:
predictor.evaluate(drug_reviews_test)

In [ ]:
predictions = predictor.predict(drug_reviews_test)

In [ ]:
predictions.value_counts()

In [ ]:
ConfusionMatrixDisplay.from_predictions(drug_reviews_test['target'], predictions)